In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as pltx
import seaborn as sns
import cv2

train_dir = '../input/herbarium-2022-fgvc9/train_images/'
test_dir = '../input/herbarium-2022-fgvc9/test_images/'

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

# JSON -> Dataframe

In [2]:
image_ids = [image["image_id"] for image in train_meta["images"]]
image_dirs = [train_dir + image['file_name'] for image in train_meta["images"]]
category_ids = [annotation['category_id'] for annotation in train_meta['annotations']]
genus_ids = [annotation['genus_id'] for annotation in train_meta['annotations']]

test_ids = [image['image_id'] for image in test_meta]
test_dirs = [test_dir + image['file_name'] for image in test_meta]

#Create the initial training dataframe with the above defined columns
train_df = pd.DataFrame({
    "image_id" : image_ids,
    "image_dir" : image_dirs,
    "category" : category_ids,
    "genus" : genus_ids})

#Create a testing dataframe
test_df = pd.DataFrame({
    "test_id" : test_ids,
    "test_dir" : test_dirs
})

# Mapping genus and family

In [3]:
#Add a genus column to the dataframe
genus_map = {genus['genus_id'] : genus['genus'] for genus in train_meta['genera']}
train_df['genus'] = train_df['genus'].map(genus_map)

##Create a family column in the datagframe based on the genus names
    # Step 1: Create dictionary of genus -> family mapping
genus_family_map = {}
for category in train_meta["categories"]:
    genus = category['genus']
    family = category['family']
    genus_family_map[genus] = family

    # Step 2: Create new column with default value of None™
train_df['family'] = None

    # Step 3: Update values in new column based on genus -> family mapping
for i, row in train_df.iterrows():
    genus = row['genus']
    if genus in genus_family_map:
        family = genus_family_map[genus]
        train_df.at[i, 'family'] = family

train_df

,image_id,image_dir,category,genus,family
0,00000__001,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
1,00000__002,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
2,00000__003,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
3,00000__004,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
4,00000__005,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
...,...,...,...,...,...
839767,15504__032,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839768,15504__033,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839769,15504__035,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839770,15504__036,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae


# Filtering to Poaceae

In [4]:
#Filter only the images of plants that are in the Poaceae family
train_df = train_df.loc[train_df['family'] == 'Poaceae']
#Reset index
train_df = train_df.reset_index(drop=True)
train_df

,image_id,image_dir,category,genus,family
0,00333__001,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
1,00333__002,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
2,00333__003,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
3,00333__004,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
4,00333__005,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
...,...,...,...,...,...
53542,15501__101,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53543,15501__103,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53544,15501__105,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53545,15501__106,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae


In [5]:
#Add category_id and species column
train_df["species"] = None

# Extract category_id and species values from categories where the family is Poaceae
species_list = []
for category in train_meta["categories"]:
    if category["family"] == "Poaceae":
        species_list.append({
            "category": category["category_id"],
            "species": category["species"]
        })

# loop through data frame and species list to update species column
for i, row in train_df.iterrows():
    for species in species_list:
        if row['category'] == species['category']:
            train_df.at[i, 'species'] = species['species']
            
train_df

,image_id,image_dir,category,genus,family,species
0,00333__001,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae,blasdalei
1,00333__002,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae,blasdalei
2,00333__003,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae,blasdalei
3,00333__004,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae,blasdalei
4,00333__005,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae,blasdalei
...,...,...,...,...,...,...
53542,15501__101,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae,bulbosa
53543,15501__103,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae,bulbosa
53544,15501__105,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae,bulbosa
53545,15501__106,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae,bulbosa


In [6]:
print('Top 15 Genus in Poaceae family')
print("")
print(train_df['genus'].value_counts())

Top 15 Genus in Poaceae family

Muhlenbergia       4228
Paspalum           3124
Poa                2608
Dichanthelium      2474
Sporobolus         2304
                   ... 
Ptilagrostiella      14
Rhipidocladum        11
Dupontia             10
Kalinia              10
Barkworthia           8
Name: genus, Length: 158, dtype: int64


# Data visualization

Genus

In [7]:
#genus_data = train_df['genus'].value_counts().head(15)
#genus_data = pd.DataFrame({'Genus' : genus_data.index,
#                     'values' : genus_data.values})
                     
#plt.figure(figsize = (20, 10))
#sns.barplot(x='values', y = 'Genus', data = genus_data , palette='summer_r')
#plt.show()

##From most to least: Muhlenbergia, Paspalum, Poa, Dichanthelium, Sporobolus, Eragrostis etc.

In [8]:
#Muhlenbergia data
muh_pas_df = train_df[(train_df['genus'] == 'Paspalum') | (train_df['genus'] == 'Muhlenbergia')]
muh_pas_df = muh_pas_df.reset_index(drop=True)
muh_pas_df

,image_id,image_dir,category,genus,family,species
0,09492__001,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
1,09492__003,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
2,09492__004,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
3,09492__005,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
4,09492__006,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
...,...,...,...,...,...,...
7347,10398__026,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae,wrightii
7348,10398__029,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae,wrightii
7349,10398__030,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae,wrightii
7350,10398__031,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae,wrightii


**Species**

In [9]:
#data_species = data_muhlenbergia['species'].value_counts().head(15) #data_species = pd.DataFrame({'Species' : data_species.index,
#                     'values' : data_species.values})#plt.figure(figsize = (20, 10))#sns.barplot(x='values', y = 'Species', data = data_species , palette='summer_r')#plt.show()#data_species

# Image displaying

In [10]:
def show_images(genus):
    images = train_df.loc[train_df['genus'] == genus]['image_dir'][:9]
    i = 1
    fig = plt.figure(figsize = (18, 18))
    plt.suptitle(genus, fontsize = '30')
    for image in images:
        img = cv2.imread(image)
        ax = fig.add_subplot(3, 3, i)
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.set_axis_off()
        i += 1
    plt.show()

# Splitting the dataset into training and validation

**2 Genuses**

In [11]:
muh_df = muh_pas_df[muh_pas_df["genus"] == "Muhlenbergia"] 
pas_df = muh_pas_df[muh_pas_df["genus"] == "Paspalum"]

#15 percent of images will be used for validation
# Muh total: 4228 --> 15% = 634
# Pas total: 3124 --> 15% = 467
muh_valid = muh_df.sample(n=634, random_state=42)
muh_train = muh_df.drop(muh_valid.index)
muh_valid = muh_valid.reset_index(drop=True)
muh_train = muh_train.reset_index(drop=True)

pas_valid = pas_df.sample(n=467, random_state=42)
pas_train = pas_df.drop(pas_valid.index)
pas_valid = pas_valid.reset_index(drop=True)
pas_train = pas_train.reset_index(drop=True)

# Merging the Muhlanbergia and Paspalum databases
muh_pas_train = pd.concat([muh_train, pas_train])
muh_pas_train = muh_pas_train.reset_index(drop=True)

muh_pas_valid = pd.concat([muh_valid, pas_valid])
muh_pas_valid = muh_pas_valid.reset_index(drop=True)

muh_pas_train

,image_id,image_dir,category,genus,family,species
0,09492__001,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
1,09492__003,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
2,09492__004,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
3,09492__005,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
4,09492__006,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae,alopecuroides
...,...,...,...,...,...,...
6246,10398__026,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae,wrightii
6247,10398__029,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae,wrightii
6248,10398__030,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae,wrightii
6249,10398__031,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae,wrightii


**Whole Poaceae family**

# Creating the model

In [12]:
import torch
import torchvision
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn import preprocessing
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [13]:
batch_size = 128
epochs = 5
IM_SIZE = 224

X_Train, Y_Train = train_df["image_dir"].values, train_df["category"].values

le = preprocessing.LabelEncoder()
train_labels = le.fit_transform(Y_Train)
train_labels = torch.as_tensor(train_labels)

X_Valid, Y_Valid = train_df["image_dir"].values, train_df["category"].values

leV = preprocessing.LabelEncoder()
val_labels = le.fit_transform(Y_Valid)
val_labels = torch.as_tensor(val_labels)

Transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# X_Train, train_labels
# X_Valid, val_labels

In [14]:
class GetData(Dataset):
    def __init__(self, FNames, Labels, Transform):
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):       
        x = Image.open(self.fnames[index])
    
        if "train" in self.fnames[index]:             
            return self.transform(x), self.labels[index]
        elif "test" in self.fnames[index]:            
            return self.transform(x), self.fnames[index]

                
trainset = GetData(X_Train, train_labels, Transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

valset = GetData(X_Valid, val_labels, Transform)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=True)

N_Classes = train_df['category'].nunique()
next(iter(trainloader))[0].shape

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = torchvision.models.densenet169(pretrained=True)


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


  0%|          | 0.00/54.7M [00:00<?, ?B/s]

In [15]:
print(model.classifier.in_features) 
print(model.classifier.out_features)

for param in model.parameters():
    param.requires_grad= False
    
n_inputs = model.classifier.in_features
last_layer = nn.Linear(n_inputs, N_Classes)
model.classifier = last_layer
if torch.cuda.is_available():
    model.cuda()
print(model.classifier.out_features)    

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters())

1664
1000
964


In [16]:
from tqdm import tqdm

true_labels = []
pred_labels = []

def train(trainloader, model, criterion, optimizer, scaler, device=torch.device("cpu")):
    train_acc = 0.0
    train_loss = 0.0
    for images, labels in tqdm(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=True):
            output = model(images)
            loss = criterion(output, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            acc = ((output.argmax(dim=1) == labels).float().mean())
            train_acc += acc
            train_loss += loss
            
    return train_acc/len(trainloader), train_loss/len(trainloader)

In [17]:
## Normal Evaluation
def evaluate(testloader, model, criterion, device=torch.device("cpu")):
    eval_acc = 0.0
    eval_loss = 0.0
    for images, labels in tqdm(testloader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            output = model(images)
            loss = criterion(output, labels)
        acc = ((output.argmax(dim=1) == labels).float().mean())
        eval_acc += acc
        eval_loss += loss
  
    return eval_acc/len(testloader), eval_loss/len(testloader)

In [18]:
%%time
from sklearn.metrics import classification_report

scaler = torch.cuda.amp.GradScaler(enabled=True)
for epoch in range(epochs):
    train_acc, train_loss = train(trainloader, model, criterion, optimizer, scaler, device=device)
    eval_acc, eval_loss = evaluate(valloader, model, criterion, device=torch.device("cuda"))

    # calculate F1 score
    with torch.no_grad():
        model.eval()
        preds = []
        targets = []
        for images, labels in tqdm(valloader):
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            preds.append(output.argmax(dim=1).cpu().numpy())
            targets.append(labels.cpu().numpy())
        preds = np.concatenate(preds)
        targets = np.concatenate(targets)
        report = classification_report(targets, preds)
        print(report)

    print("") 
    print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss}")
    print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss}")
    print("===="*8) 

100%|██████████| 419/419 [20:48<00:00,  2.98s/it]
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.75      0.23      0.35        13
           2       0.18      0.39      0.25        23
           3       0.00      0.00      0.00        30
           4       0.06      0.78      0.12        74
           5       0.00      0.00      0.00        50
           6       0.50      0.05      0.09        42
           7       0.00      0.00      0.00        10
           8       0.67      0.11      0.19        71
           9       0.00      0.00      0.00        77
          10       0.00      0.00      0.00        39
          11       0.00      0.00      0.00        35
          12       0.24      0.12      0.16        73
          13       0.42      0.10      0.17        77
          14       0.43      0.21      0.28        77
          15       1.00      0.05      0.10        39
          16       0.04      0.46      0.07        48
          17       1.00    

100%|██████████| 419/419 [19:30<00:00,  2.79s/it]
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.33      0.50         9
           1       0.06      1.00      0.12        13
           2       0.21      0.39      0.27        23
           3       0.42      0.67      0.51        30
           4       0.51      0.53      0.52        74
           5       0.00      0.00      0.00        50
           6       1.00      0.14      0.25        42
           7       0.62      0.80      0.70        10
           8       0.28      0.49      0.36        71
           9       1.00      0.05      0.10        77
          10       1.00      0.03      0.05        39
          11       1.00      0.09      0.16        35
          12       0.32      0.45      0.38        73
          13       0.62      0.17      0.27        77
          14       0.66      0.27      0.39        77
          15       1.00      0.13      0.23        39
          16       1.00      0.08      0.15        48
          17       0.81    

 77%|███████▋  | 321/419 [14:05<04:18,  2.64s/it]


TypeError: int() argument must be a string, a bytes-like object or a number, not 'JpegImageFile'